<a href="https://colab.research.google.com/github/Sep-eg/SubjectClassifier-usingYoutubeSubtitle/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AI BootCamp Project

네이버 지식인을 이용하여 질문글을 작성하거나, 당근마켓을 통해 중고거래 글을 작성하려 할 때 등 내가 입력한 내용을 기반으로 자동으로 카테고리를 추천해주는 경험을 해본적이 있을 것이다. 

이러한 기능은 글을 작성하는 작성자가 특별히 신경쓰지 않더라도 내용을 검색해보고 싶은 검색자가 쉽게 찾을 수 있도록 카테고리 분류작업을 대신 해줄수 있다.

하지만 만약 영상데이터라면 그 분류를 어떻게 진행할 수 있을까? 나는 이 고민을 유튜브 자막 데이터를 이용하여 해결해 보기로하였다. 

## 데이터 불러오기 및 전처리

데이터는 kaggle에 있는 Math Lectures(https://www.kaggle.com/extralime/math-lectures) 를 활용하였다.

### 데이터 불러오기

In [ ]:
# # 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt

from nltk.corpus import stopwords

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
BASE_PATH = '/content/drive/MyDrive/sec4project/'
df = pd.read_csv(BASE_PATH+'raw_text.csv')

In [ ]:
df.head()

In [ ]:
df.shape

데이터는 860개의 영상의 subtitle text와 어떤 분야의 영상인지를 나타내는 label을 가지고 있다.

### 데이터 전처리

In [ ]:
# 결측치확인 -> 없음
df.isna().sum()

In [ ]:
# text와 label 분리
texts = df['text']
label = df['label']

In [ ]:
# 중복데이터 확인
texts.duplicated().sum()

중복된 데이터가 한 쌍 발견되었으므로 어떤것을 지워야할지 확인한다.

In [ ]:
# 중복된 데이터의 인덱스 확인
texts[texts.duplicated()==True].index

In [ ]:
# 중복된 데이터와 같은 데이터가 위치한 인덱스 확인
sum(texts.drop_duplicates(keep='first').index) - sum(texts.drop_duplicates(keep='last').index)

In [ ]:
# 탐색한 인덱스에 위치한 데이터가 중복이 맞는지 확인
texts[443] == texts[408]

In [ ]:
# 두데이터 레이블 확인
label[443], label[408]

In [ ]:
texts[443]

강의 내용이 Probability에 관한 내용으로 판단되므로 CS레이블로 분류된 408번 인덱스의 데이터를 제거한다.

In [ ]:
# 인덱스 443 제거
texts.drop(index=443, inplace=True)
label.drop(index=443, inplace=True)
texts.shape, label.shape

텍스트 데이터 시작 부분에 같은내용이 반복되는 부분이 있으므로 삭제를 위해 데이터 확인

In [ ]:
texts[0]

In [ ]:
texts[3]

The following content ... ocw.mit.edu가 반복적으로 들어가는 것을 확인 해당부분 제거

In [ ]:
"""
The following content ... ocw.mit.edu.
로 중복되는 구간을 제거하는 함수
"""

def cut_Useless(text):

    if (text.startswith("The following content") == True
        or text.startswith("The following\ncontent")==True): 
        #ocw.mit.edu. 를 기준으로 split하고 앞부분을 스킵해서 제거 
        for idx, sentence in enumerate(text.split("ocw.mit.edu. ")):
            if idx==0:
                continue
            return sentence

    return text

In [ ]:
texts = texts.apply(cut_Useless)

본격적인 전처리를 진행하기전 대부분의 text가 "\n"을 구분기호로 사용하고 있는것으로 보이지만 그렇지 않은경우 문자열을 분리를 할때 문제가 생길 수 있으므로 없는 text가 있는지 확인

In [ ]:
texts_len = len(texts)
for idx, text in enumerate(texts):
    if "\n" in text == False:
        print(idx)
        continue
    texts_len -= 1

if texts_len == 0:
    print("\\n을 포함하지 않은 text는 없습니다.")

In [ ]:
texts[0]

---
text를 확인해보면 자막데이터인 관계로 "\n"은 문장의 끝과는 상관없는 것을 확인할 수 있다.

1. 따라서 "\n"은 공백으로 대체하고 추후 "."를 기준으로 split 진행

2. 또한 문장을 나누는 기준으로 "."을 사용하기 때문에 "?", "!"또한 "."으로 대체한다.

3. 영상의 주제에 영향을 끼치는건 구체적인 계산식 보다는 중요 키워드의 영향이 크다고 생각되기 때문에 영문자를 제외한 텍스트는 제거한다.

4. 리소스를 줄이기위해 주제를 판단하는데 미치는 영향이 적은 불용어는 제거한다.

In [ ]:
"""
특수문자 대체, 불용어 삭제 등 전반적인 전처리 함수
"""

def preprocessing(text, remove_stopwords=True):

    text=text.replace("\n", " ").replace("!", ".").replace("?", ".")
    
    text=re.sub("[^a-zA-Z.:]", " ", text)
    # ":"은 말하는 화자를 구분하는 의미로 사용되고 있으므로 의미를 살리기위해 제거하지 않는다.
    
    words=text.lower().split()
    
    if remove_stopwords:
        # 불용어 제거
        
        # 영어 불용어 불러오기
        stops=set(stopwords.words("english"))

        # 주로 수학, CS에 관한 내용이므로 변수등으로 많이 사용될 수 있는 알파벳 모음을 불용어에 추가
        stops.update(['a','b','c','d','e',
                      'f','g','h','i','j',
                      'k','l','m','n','o',
                      'p','q','r','s','t',
                      'u','v','w','x','y','z'])
        
        # 불용어가 아닌 단어로 이뤄진 새로운 리스트 생성
        words=[w for w in words if not w in stops]
        # 5. 단어 리스트를 공백을 넣어서 하나의 글로 합친다.
        text=' '.join(words)
    
    # 불용어를 제거하지 않을 때
    else:
        text=' '.join(words)
    
    return text

In [ ]:
texts = texts.apply(preprocessing)

In [ ]:
texts[0]

###데이터 분리
본격적으로 모델을 만들기 전 데이터를 train set과 test set으로 나누겠습니다.

프로젝트의 재현성을 위해 이후 진행할 모든 작업에서 random_state는 11로 지정합니다.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(texts, label,
                                                    train_size=0.8, stratify=label,
                                                    random_state=11)

In [ ]:
X_train, X_test

### 레이블 변경
현재의 문자형태로 되어있는 레이블은 추후 모델을 학습시킬때 장애가 될수있으므로 숫자형 데이터로 매칭하여 변환시킨다.

In [ ]:
label_name = y_train.unique()

In [ ]:
for idx, tmp_label in enumerate(label_name):
    y_train.replace(tmp_label, idx, inplace=True)
    y_test.replace(tmp_label, idx, inplace=True)

In [ ]:
y_train.unique(), y_test.unique()

## 모델링

### LSTM을 이용한 간단한모델

In [ ]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
tokenizer = Tokenizer() # 리소스 한계로인해 단어개수는 15000개로 제한한다.
tokenizer.fit_on_texts(X_train)
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

In [ ]:
# train_sequences중 가장 긴 sequence의 길이를 max_len으로 지정
max_len = 0
for sequen in train_sequences:
    if max_len < len(sequen):
        max_len = len(sequen)

In [ ]:
# embedding을 위해 token화한 단어들의 개수를 vocab_size로 저장
word_to_index = tokenizer.word_index
vocab_size = len(word_to_index) + 1
embedding_size = 300

In [ ]:
# sequence들을 패딩하는 과정, 모든 sequence의 길이를 맞춰준다.
train_sequences = pad_sequences(train_sequences, maxlen=max_len)
test_sequences = pad_sequences(test_sequences, maxlen=max_len)

# 11개의 노드를 통해 분류되기 때문에 label을 범주형으로 변환
y_train_cate = to_categorical(y_train, num_classes=11)
y_test_cate = to_categorical(y_test, num_classes=11)

In [ ]:
from tensorflow.keras.layers import LSTM, Embedding, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization, Flatten, concatenate, InputLayer
from tensorflow.keras.models import Sequential, Model

In [ ]:
tf.random.set_seed(11)
model = Sequential()
model.add(Embedding(vocab_size, embedding_size))
model.add(LSTM(128))
model.add(BatchNormalization())
model.add(Dense(11, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
history1 = model.fit(train_sequences, y_train_cate, epochs=6, batch_size=32, validation_data=(test_sequences, y_test_cate))

In [ ]:
acc = history1.history['accuracy']
val_acc = history1.history['val_accuracy']

sns.lineplot(x=range(len(acc)), y=acc)
sns.lineplot(x=range(len(acc)), y=val_acc)
plt.title('LSTM_model accuracy')
plt.legend(['acc', 'val_acc']);

LSTM을 이용해 간단한 모델을 구현해 보았을때, train set에 대해서 5epoch만에 정확도가 1.0으로 높게 나타나는 모습을 보였다.

하지만 이는 train set에 과적합된 결과로 test set에 대해서는 정확도가 전혀 개선되지 않는 모습을 보였는데 이는 강의자막 데이터라는 특성상

강사, 혹은 세부주제에 따라 내용의 흐름이 크게 달라질 수 있기 때문에 LSTM의 순차적 처리방식이 맞지 않은것으로 보았다.

이에 따라 모든 문서의 단어를 확인하여 중요 단어를 판단하는 TF-IDF를 이용한 분류를 진행해 보기로 하였다.

In [ ]:
# 학습된 모델 저장
# model.save(BASE_PATH+"LSTM_model.h5")

### TF-IDF과정을 거친 데이터를 이용한 분류

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vector = TfidfVectorizer()
train_tfidf = vector.fit_transform(X_train).toarray()
test_tfidf = vector.transform(X_test).toarray()

In [ ]:
max_features = train_tfidf.shape[1]

In [ ]:
tf.random.set_seed(11)

model2 = Sequential()
model2.add(InputLayer(input_shape=(max_features)))
model2.add(Dense(256, activation='relu'))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(11, activation='softmax'))

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
history2 = model2.fit(train_tfidf, y_train_cate, epochs=6, batch_size=8, validation_data=(test_tfidf, y_test_cate))

In [ ]:
acc = history2.history['accuracy']
val_acc = history2.history['val_accuracy']

sns.lineplot(x=range(len(acc)), y=acc)
sns.lineplot(x=range(len(acc)), y=val_acc)
plt.title('tfidf_fully_model accuracy')
plt.legend(['acc', 'val_acc']);

In [ ]:
# 학습된 모델 저장
# model2.save(BASE_PATH+"tfidf_fully_model.h5")

TF-IDF를 통해 얻은 데이터를 이용한 학습도 마찬가지의 결과였다. 

train set에 대한 정확도는 빠르게 1.0을 향해 수렴해갔지만,

test set에 대한 정확도는 여전히 거듭되는 학습에도 개선되지 못했다.

(그래프는 6번의 epoch만을 나타냈지만 epoch수를 높여도 마찬가지였다.)

### 새로운 접근의 필요(BERT)
이러한 결과가 나타난 다양한 이유가 있을 수 있겠지만, 근본적인 데이터셋 부족이 가장 큰 이유로 보인다.

당장 더 큰 데이터를 모으는 것은 어려우므로 이를 개선해보기 위하여 사전학습 모델 BERT에 fine-tuning을 적용하여 학습을 진행해 보았다.

In [ ]:
# !pip install -q -U tensorflow-text

In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
bert_model_name = 'experts_wiki_books' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [ ]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=False, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = Dropout(0.3)(net)
    net = Dense(256, activation='relu')(net)
    net = Dense(11, activation='softmax', name='classifier')(net)
    return tf.keras.Model(text_input, net)

In [ ]:
bert_model = build_classifier_model()

In [ ]:
bert_model.summary()

In [ ]:
tf.random.set_seed(11)
bert_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
# 모델에 preprocessing과정이 포함되어 있으므로 Tokenizer를 거치지않은 데이터를 활용하여 학습
history3 = bert_model.fit(X_train, y_train_cate, epochs=50, batch_size=64, validation_data=(X_test, y_test_cate))

In [ ]:
acc = history3.history['accuracy']
val_acc = history3.history['val_accuracy']

sns.lineplot(x=range(len(acc)), y=acc)
sns.lineplot(x=range(len(acc)), y=val_acc)
plt.title('with_BERT_model accuracy')
plt.legend(['acc', 'val_acc']);

In [ ]:
# 50 epoch동안 test set 평균 정확도
sum(history3.history['val_accuracy'])/50

In [ ]:
loss = history3.history['loss']
val_loss = history3.history['val_loss']

sns.lineplot(x=range(len(acc)), y=loss)
sns.lineplot(x=range(len(acc)), y=val_loss)
plt.title('with_BERT_model loss')
plt.legend(['loss', 'val_loss']);

In [ ]:
# bert_model.save(BASE_PATH+"with_BERT_model.h5")

experts_wiki_books기반의 Bert를 Fine Tuning하여 학습하였다.

학습을 진행할수록 train set에 대한 loss는 감소하고 정확도는 증가하는 모습을 보였다.

하지만, 다른 모델들과 마찬가지로 test set의 loss는 꾸준히 증가하는 경향을 나타냈고,

정확도는 특별한 경향성을 보이지 않고 오르락 내리락 하는 형태만 계속해서 나타냈다.

이에 더 이상의 학습은 무의미하다고 판단하여 50epoch까지만 학습을 진행하였다.

## RandomForest를 이용한 예측 및 결론

### TF-IDF + RandomForest
그렇다면 머신러닝 방법론을 적용해보면 어떨까? 하는 생각을 가지게 되었다.

머신러닝에 데이터를 학습시키기 위해서는 데이터의 특성을 추출해낼 필요가 있는데,

이를 위해 앞에서 사용했던 TF-IDF분류를 통해 생성한 특성을 사용하여 학습을 진행하였다.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(random_state=11)

In [ ]:
rfc.fit(train_tfidf, y_train)

In [ ]:
rfc.score(train_tfidf, y_train)

In [ ]:
rfc.score(test_tfidf, y_test)

그 결과는 아무런 튜닝을 거치지 않은 기본적인 모델임에도 불구하고, 

train set 정확도 1.0 / test set 정확도 0.1511로 나타났다.

이 결과는 앞에서 시도했던 딥러닝 방법론들과 유사한 성능으로 모델을 학습시키는데 필요한 리소스, 시간등을 고려했을때 오히려 더 나은 선택일 수 있다.

하지만 결과적으로 시험해본 모든 모델은 train set에 과적합되고 test set에 대해서는 전혀 예측하지 못하는 모습을 보였다.

그 이유로는 다양한것이 존재할 수 있겠지만, 

